Machine Learning Feature Extraction Optimization Using Evolutionary Computation Algorithms


Problem Definition

Dataset Explaination

Strategies


**Possible datasets**

[Heart Failure Prediction Dataset](https://www.kaggle.com/datasets/andrewmvd/heart-failure-clinical-data) (13 features)

[Real or Fake Jobposting Prediction Dataset](https://www.kaggle.com/datasets/shivamb/real-or-fake-fake-jobposting-prediction) (18 features)

[Footballer Price Prediction Dataset](https://www.kaggle.com/datasets/rashadrmammadov/football-teams-price-prediction) (26 features)

[Loan Default Prediction Dataset](https://www.kaggle.com/datasets/hemanthsai7/loandefault) (35 features)

[Bankruptcy Prediction Dataset](https://www.kaggle.com/datasets/fedesoriano/company-bankruptcy-prediction) (96 features)


In [7]:
import pandas as pd
import numpy as np

Random Forest

XGBoost

Logistic Regression

Linear Regression

FCNN(?)

SVM(?)

In [ ]:
!pip install kagglehub

In [4]:
import kagglehub


path_bankrupt = kagglehub.dataset_download("fedesoriano/company-bankruptcy-prediction")
path_loan = kagglehub.dataset_download("hemanthsai7/loandefault")
path_footballer = kagglehub.dataset_download("rashadrmammadov/football-teams-price-prediction")
path_job = kagglehub.dataset_download("shivamb/real-or-fake-fake-jobposting-prediction")
path_heart = kagglehub.dataset_download("andrewmvd/heart-failure-clinical-data")


/home/tugberk/.cache/kagglehub/datasets/fedesoriano/company-bankruptcy-prediction/versions/2


In [5]:
heart_data_path = f"{path_heart}/heart_failure_clinical_records_dataset.csv"
job_data_path = f"{path_job}/fake_job_postings.csv"
footballer_data_path = f"{path_footballer}/Football_teams_price_data.csv"
loan_data_path = f"{path_loan}/train.csv"
bankrupt_data_path = f"{path_bankrupt}/data.csv"


In [10]:
heart_data = pd.read_csv(heart_data_path)
job_data = pd.read_csv(job_data_path)
footballer_data = pd.read_csv(footballer_data_path)
loan_data = pd.read_csv(loan_data_path)
bankrupt_data = pd.read_csv(bankrupt_data_path)

13

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def fitness(chromosome, model, X, y):
    # Select features based on the chromosome
    selected_features = np.where(chromosome == 1)[0]
    if len(selected_features) == 0:
        # Return a very low fitness score if no features are selected
        return 0.0

    # Subset the dataset using selected features
    X_selected = X[:, selected_features]

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

    try:
        # Train the model
        model.fit(X_train, y_train)

        # Predict on the test set
        y_pred = model.predict(X_test)

        # Calculate and return accuracy
        fitness = accuracy_score(y_test, y_pred)
        return fitness
    except Exception as e:
        print(f"Error during model training or prediction: {e}")
        return 0.0


In [ ]:
def parent_selection(f_values, n_pop): #parent selection for crossover
    f_sum = np.sum(f_values)
    probabilities = f_values / f_sum
    return np.random.choice(np.arange(n_pop), size = 2, p = probabilities) #selecting 2 parents (size = 2) for crossover

In [ ]:
def crossover(population, parents, crossover_rate, dimensions): #single-point crossover
    if np.random.randint() < crossover_rate:
        point = np.random.randint(1, dimensions)
        child1 = np.concatenate((population[parents[0], :point], population[parents[1], point:]))
        child2 = np.concatenate((population[parents[1], :point], population[parents[0], point:]))
        return child1, child2
    else:
        return population[parents[0]], population[parents[1]]

In [ ]:
def bitFlip_mutation(chromosome, mutation_rate): #bit flip mutation
    for i in range(len(chromosome)):
        if np.random.rand() < mutation_rate:
            chromosome[i] = 1 - chromosome[i]
    return chromosome

In [ ]:
def genetic_algorithm(dataset, model, dimension, population_size, generations, mutation_rate, crossover_rate):
    pop = np.random.randint(0, 2, size=(population_size, dimension))
    X = dataset.iloc[:, :-1].values  # All columns except the last one
    y = dataset.iloc[:, -1].values 
    fitness_values = [fitness(chromosome, model, X, y) for chromosome in pop]
    max = np.argmax(fitness_values)
    best_chromosome = pop[max]
    best_fitness = fitness_values[max]
    print(f"generation:{0}, best fitness: {best_fitness}, best chromosome: {best_chromosome}")
    for gen in range(1, generations):
        pop_new = np.zeros((len(pop), dimension), dtype=int)
        for i in range(len(pop) // 2):
            parents = parent_selection(fitness_values, len(pop))
            child1, child2 = crossover(pop, parents, crossover_rate, dimension)
            child1 = bitFlip_mutation(child1, mutation_rate)
            child2 = bitFlip_mutation(child2, mutation_rate)
            pop_new[i*2] = child1
            pop_new[i*2 + 1] = child2
        pop = pop_new
        fitness_values = [fitness(chromosome, model, X, y) for chromosome in pop]
        
        max = np.argmax(fitness_values)
        if fitness_values[max] > best_fitness:
            best_chromosome = pop[max]
            best_fitness = fitness_values[max]
        print(f"generation:{gen}, best fitness: {best_fitness}, best chromosome: {best_chromosome}")
    

In [ ]:
#write me a differential_evolution function from the next cell

def differential_evolution(dataset, model, dimension, population_size, generations, mutation_rate, crossover_rate):
    population = np.random.rand(population_size, dimension)
    for i in range(generations):
        for j in range(population_size):
            x = np.random.choice(population_size, 3, replace=False)
            a, b, c = population[x]
            mutant = a + mutation_rate * (b - c)
            crossover = np.random.rand(dimension) < crossover_rate
            population[j] = np.where(crossover, mutant, population[j])
    return population

In [ ]:
#write me a evolution_strategy function from the next cell

def evolution_strategy(dataset, model, dimension, population_size, generations, mutation_rate, crossover_rate):
    population = np.random.rand(population_size, dimension)
    for i in range(generations):
        for j in range(population_size):
            x = np.random.choice(population_size, 3, replace=False)
            a, b, c = population[x]
            mutant = a + mutation_rate * (b - c)
            crossover = np.random.rand(dimension) < crossover_rate
            population[j] = np.where(crossover, mutant, population[j])
    return population

In [ ]:
#Algorithms : GA, DE, ES
#Datasets: heart_data(13 dimension), job_data(18 dimension), footballer_data(26 dimension) 
#          loan_data(35 dimension), bankrupt_data(96 dimension)
def evolution(algorithm, dataset, model, pop_size, gens = 10, mut_rate = 1/len(dataset.columns), cross_rate = 0.9):

    dimension = len(dataset.columns)
    population_size = pop_size
    generations = gens
    mutation_rate = mut_rate
    crossover_rate = cross_rate

    if algorithm == "GA":
        genetic_algorithm(dataset, model, dimension, population_size, generations, mutation_rate, crossover_rate)
    elif algorithm == "DE":
        differential_evolution(dataset, model, dimension, population_size, generations, mutation_rate)
    elif algorithm == "ES":
        evolution_strategy(dataset, model, dimension, population_size, generations, mutation_rate)